In [7]:
# automatically reload dependant notebooks
%load_ext autoreload
%autoreload 2
import import_ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# All-Pairs Shortest Paths

Given a directed, weighted graph $G = (V, E)$, All-Pairs Shortest Path (ASP) algorithm discovers the shortest paths between each pair of vertices. See CLRS 4ed Chapter 23 *All-Pairs Shortest Paths* p.646. ASP can be used to tabulate the cheapest airfares between pairs of city. The ticket price from city $C$ to $D$ is not necessarily the same as that from $D$ to $C$, due to day and time of travel, airlines in operation, flight availability, stopovers, etc.

## ASP graph with adjacency matrix of edge weights

Unlike those algorithms in the earlier chapters, ASP algorithms use dense graphs represented with adjacency matrices. So, we derive `ASPGraph` from `MtxGraph`. See Equation 23.1 p.647 for the description of adjacency matrix representation. Note that we use `WgtEdge` defined in the [mst.ipynb](./mst.ipynb) notebook.

In [8]:
from graph import *
from mst import *
from asp import *
from util import *

class ASPGraph(MtxGraph):
  def __init__(self, tag: Tag):
    super().__init__(tag)

  def makeVEw(self, vt: [Tag], et: [Tag], ew: {Tag, float}) -> None:
    self.makeV(vt)
    self.makeEw(et, ew)
  def makeEw(self, et: [Tag], ew: {Tag, float}) -> None:
    for etag in et:
      [utag, vtag] = parseETag(etag)
      e = WgtEdge(self.getV(utag), self.getV(vtag), ew[etag])
      self.ee[e.tag] = e
      [i, j] = indicesOfETag(etag)
      self.ww[i][j] = ew[etag]

## Floyd_Warshall ASP algorithm

Floyd_Warshall ASP algorithm employs dynamic programming to find the optimal (shortest) paths between every pair of vertices. See 23.2 The Floyd-Warshall algorithm p.655. The runtime of the Floyd-Warshall algorithm is $\Theta(V^3)$. See p.657.

[Dynamic programming](https://en.wikipedia.org/wiki/Dynamic_programming) is a tabulation-based optimisation technique. [Greedy algorithms](https://en.wikipedia.org/wiki/Greedy_algorithm), like Kruskal's MST, Prim's MST, and Dijkstra's SSP, select at each optimisation step a locally optimal choice. But dynamic programming algorithms select at each optimisation step the globally optimal choice. See Chapter 14 Dynamic Programming p.362.

Do not confuse dynamic programming with [dynamic programming languages](https://en.wikipedia.org/wiki/Dynamic_programming_language)—like Python.

In [9]:
def aspFloydWarshall(g: ASPGraph) -> [WMtx, WMtx]:
  n = g.numVV()
  r = range(0, n)
  np1 = n + 1
  # initialize
  g.dd = [[]] * np1
  g.dd[0] = g.ww
  g.pp = [[]] * np1
  g.pp[0] = [[]] * n
  for i in r:
    g.pp[0][i] = [-Infinity] * n  # use -Infinity instead of NIL as used in CLRS; see p.659
    for j in r:
      g.pp[0][i][j] = -Infinity if i == j or g.ww[i][j] == Infinity else i
  # discover ASP in graph g
  for k in range(1, np1):
    g.dd[k] = [[]] * n
    g.pp[k] = [[]] * n
    for i in r:
      g.dd[k][i] = [Infinity] * n
      g.pp[k][i] = [-Infinity] * n
      for j in r:
        km1 = k - 1
        g.dd[k][i][j] = min(g.dd[km1][i][j], g.dd[km1][i][km1] + g.dd[km1][km1][j])
        g.pp[k][i][j] = g.pp[km1][km1][j] if g.dd[km1][i][j] > g.dd[km1][i][km1] + g.dd[km1][km1][j] else g.pp[km1][i][j]  # see Equation 23.8 p.659
  return g.dd[n], g.pp[n]

# Conclusion

In this notebook, we implemented Floyd-Warshall ASP algorithm for dense graphs, transitive closure of a directed graph, and Johnson's ASP algorithm for sparse graphs. Tests for these ASP algorithms are implemented in [`asptest.ipynb`](./asptest.ipynb).